In [23]:
import requests
import json
import re
import urllib.parse
import urllib3
from concurrent.futures import ThreadPoolExecutor
import time

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)



In [24]:
# Base URL
base_url_template = (
    "https://www.jomashop.com/graphql?operationName=category&variables={variables}&extensions={extensions}"
)

# Headers for requests
headers = {
    "Host": "www.jomashop.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
}

# JSON templates
variables_template = {
    "id": 871,
    "onServer": True,
    "pageSize": 60,
    "filter": {"category_id": {"eq": "871"}},
    "sort": {}
}

extensions = {
    "persistedQuery": {
        "version": 1,
        "sha256Hash": "39eb08ac6e70e0c9a72474f735320396645bf9b77352cb1041a639d00e80e6bf"
    }
}

In [ ]:
# Function to extract brand
def extract_brand(name):
    match = re.match(r'^(\S+)', name)
    return match.group(1) if match else None

# Hàm để lấy URL đầu tiên
def extract_first_url(media_gallery):
    # Kiểm tra nếu media_gallery là danh sách
    if isinstance(media_gallery, list):
        for item in media_gallery:
            # Tìm URL trong chuỗi
            urls = re.findall(r'https?://[^\s]+', str(item))
            if urls:
                return urls[0]  # Trả về URL đầu tiên
    return None  # Trả về None nếu không tìm thấy

# Hàm gửi thông tin sản phẩm lên Google Form
from concurrent.futures import ThreadPoolExecutor
import time

# Hàm gửi thông tin sản phẩm lên Google Form
def send_to_google_form(results):
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSc948XxTD7e-BqqyEk5hNr_rUFVI-n7N_Zs1R7pu6NakjhN2g/formResponse"
    
    # Hàm gửi một sản phẩm
    def send_product(product):
        payload = {
            "entry.366340186": product.get("name1"),
            "entry.291509220": product.get("brand"),
            "entry.1228184084": product.get("sale_price"),
            "entry.2032688428": product.get("discount"),
            "entry.1099633809": product.get("product_url"),
            "entry.1349079639": product.get("image_url"),
            "fvv": 1
        }
        response = requests.post(form_url, data=payload)
        return response.status_code == 200

    # Sử dụng ThreadPoolExecutor để gửi song song
    with ThreadPoolExecutor(max_workers=10) as executor:  # Tối đa 10 luồng song song
        futures = [executor.submit(send_product, product) for product in results]
        for future in futures:
            success = future.result()
            print(f"Product sent: {'Success' if success else 'Failed'}")

In [ ]:
# Loop through pages
for i in range(1, 251):  # Fetch only the first page for testing
    variables_template["currentPage"] = i

    # URL encode variables and extensions
    variables_encoded = urllib.parse.quote_plus(json.dumps(variables_template))
    extensions_encoded = urllib.parse.quote_plus(json.dumps(extensions))

    # Construct the URL
    url = base_url_template.format(variables=variables_encoded, extensions=extensions_encoded)

    # Make the request
    response = requests.get(url, headers=headers, verify=False)

    if response.status_code == 200:
        response_json = response.json()
        # Extract items
        items = response_json.get("data", {}).get("products", {}).get("items", [])
        results = []

        for item in items:
            result = {
                "id": item.get("id"),
                "sku": item.get("sku"),
                "name": item.get("name"),
                "name1" : re.sub(r'^\S+\s*', '', name),
                "brand": extract_brand(item.get("name", "")),
                "regular_price": f"${item.get('price_range', {}).get('minimum_price', {}).get('msrp_price', {}).get('value')}",
                "sale_price": f"${item.get('price_range', {}).get('minimum_price', {}).get('plp_price', {}).get('now_price')}",
                "discount": item.get("price_range", {}).get("minimum_price", {}).get("plp_price", {}).get("discount"),
                "stock_status": item.get("stock_status"),
                "product_url": f"https://www.jomashop.com/{item.get('url_key')}.html" if item.get("url_key") else None,
                "image_url": extract_first_url(item.get("media_gallery", []))  
            }
            results.append(result)
            #     # Print first 5 results
        #     for idx, res in enumerate(results[:5]):
        #         print(f"Product {idx + 1}:")
        #         print(json.dumps(res, indent=4))
        # else:
        #     print(f"Failed to fetch page {i}: Status code {response.status_code}")
        #     break
        # Đẩy toàn bộ sản phẩm của trang này lên Google Form
        send_to_google_form(results)
    else:
        print(f"Failed to fetch page {i}: Status code {response.status_code}")
        break



Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success
Product sent: Success


KeyboardInterrupt: 